# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,howard springs,-12.4970,131.0470,87.82,75,100,2.24,AU,1732586664
1,1,puerto baquerizo moreno,-0.9000,-89.6000,71.13,71,80,10.74,EC,1732586667
2,2,stanisic,45.9400,19.1653,38.79,76,6,7.02,RS,1732586668
3,3,tsiombe,-25.3000,45.4833,75.45,93,53,9.35,MG,1732586669
4,4,de queen,34.0379,-94.3413,49.01,76,0,4.61,US,1732586670


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
#%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_width = 896,
    hover_cols = ["City", "Humidity"]
)

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null values -
# max temp: 27 C ~ 80.6 F
# min temp: 21 C ~ 69.8 F
# wind < 4.5
# cloudiness = 0

idealCityDF = city_data_df.loc[
            (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] < 80.6) \
            & (city_data_df["Wind Speed"] < 4.5) \
            & (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

5


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
127,127,lazaro cardenas,17.9583,-102.2000,78.01,77,0,3.56,MX,1732586526
133,133,beroroha,-21.6667,45.1667,77.76,71,0,4.03,MG,1732586826
251,251,san pedro pochutla,15.7333,-96.4667,76.71,94,0,2.95,MX,1732587016
325,325,porto seguro,-16.4497,-39.0647,73.92,88,0,2.30,BR,1732587241
333,333,brisas de zicatela,15.8369,-97.0419,79.32,89,0,2.30,MX,1732587251


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
127,lazaro cardenas,MX,17.9583,-102.2000,77,
133,beroroha,MG,-21.6667,45.1667,71,
251,san pedro pochutla,MX,15.7333,-96.4667,94,
325,porto seguro,BR,-16.4497,-39.0647,88,
333,brisas de zicatela,MX,15.8369,-97.0419,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [48]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories": "accomadation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df

Starting hotel search
lazaro cardenas - nearest hotel: No hotel found
beroroha - nearest hotel: No hotel found
san pedro pochutla - nearest hotel: No hotel found
porto seguro - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [50]:
#%%capture --no-display

# Configure the map plot
hotelMapPlot = hotel_df.hvplot.points(
    x="Lng", 
    y="Lat",
    geo=True,
    size=5,  
    scale=1,
    color="City Name",  
    alpha=0.5,
    tiles="OSM",
    frame_width=896,
    frame_height=504,
    hover_cols=["City Name", "Hotel Name", "Hotel Address"]  
)

# Display the map
hotelMapPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Hotel Address)